In [1]:
import pandas as pd
import os, string

def shorten(name):
    items = name.split("-")
    if len(items[0]) <= 3:
        if not items[1:]:
            return items[0]
        else:
            return items[0]+"-"+reduce(lambda i, j: i+j, map(lambda i: i[0], items[1:]))
    else:
        return reduce(lambda i, j: i+j, map(lambda i: i[0], items))

In [2]:
file_list = sorted([file for root, subs, files in os.walk("../data/raw/investing") for file in files if file.endswith(".csv")])

In [3]:
tables = {}
for file in file_list:
    tables[string.join(file.replace(".csv", "").split("-")[:-1], "-")] = pd.read_csv("../data/raw/investing/{}".format(file), index_col="Datetime").sort_index()

In [4]:
kw, table = tables.popitem()
table.columns = ["{}-{}".format(col, shorten(kw)) for col in table.columns]
while tables:
    try:
        kw, other = tables.popitem()
        table = table.join(other, how="outer", rsuffix="-{}".format(shorten(kw)))
    except:
        break

In [5]:
table.reset_index().to_excel("../data/raw/investing/economic-calendars.xlsx", index=False)